# Trafic de données avec Python–pandas


Ce carnet comprends plusieurs sections :

1. Introduction et objectifs
2. Objets principaux de pandas : `Series` et `DataFrame`
3. Lire / écrire des tables de données
4. Gérer une table de données (discrétisation, recodage, dummies, etc.)
5. Statistiques descriptives élémentaires
6. Manipulation de tables (jointures, concaténations)
7. Trafic séquentiel de gros fichiers (lecture par morceaux, HDF5)

Vous pouvez compléter ce carnet en ajoutant vos propres commentaires et exercices.

## 1. Introduction

Objectif : transformer des **données brutes** en une table propre et exploitable
(`DataFrame`) pour l'analyse statistique ou l'apprentissage automatique.

On parlera souvent de *data munging* ou *data wrangling* : nettoyage,
sélection de variables, gestion des valeurs manquantes, recodage, etc.

Les exemples portent sur le jeu de données **Titanic** (Kaggle).

In [ ]:
# Importations de base
import pandas as pd
import numpy as np

pd.__version__

## 2. Objets `Series` et `DataFrame`

### 2.1 `Series`

Une `Series` est un vecteur (1D) de valeurs **indexées**. Très utilisée pour
les séries temporelles ou comme colonne d'un `DataFrame`. 

In [ ]:
# Exemple simple de Series
s = pd.Series([4, 7, -5, 3], index=["a", "b", "c", "d"])
s


### 2.2 `DataFrame`

Un `DataFrame` est une **table de données** : même index de lignes pour
plusieurs variables (colonnes) éventuellement de types différents.

In [ ]:
# Exemple de DataFrame
data = {
    "state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
    "year": [2000, 2001, 2002, 2001, 2002],
    "pop": [1.5, 1.7, 3.6, 2.4, 2.9],
}

frame = pd.DataFrame(data)
frame

In [ ]:
# Changer l'ordre des colonnes
pd.DataFrame(data, columns=["year", "state", "pop"])

In [ ]:
# Ajouter une colonne avec valeurs manquantes
frame2 = pd.DataFrame(
    data,
    columns=["year", "state", "pop", "debt"],
    index=["one", "two", "three", "four", "five"],
)
frame2

In [ ]:
# Création et suppression de variables
frame2["debt"] = 16.5
frame2["eastern"] = frame2.state == "Ohio"
display(frame2)

del frame2["eastern"]
frame2

## 3. Lire et écrire des tables de données

On utilise principalement `pd.read_csv` / `pd.read_table` pour lire des fichiers texte
et `DataFrame.to_csv` pour écrire. Les options permettent de gérer séparateurs,
noms de colonnes, types, valeurs manquantes, etc.

In [ ]:
# Exemple avec les données Titanic depuis wikistat
path = "http://www.wikistat.fr/data/"
df_test = pd.read_csv(path + "titanic-train.csv", nrows=5)
df_test

In [ ]:
# Lecture en sélectionnant certaines colonnes et noms explicites
df = pd.read_csv(
    path + "titanic-train.csv",
    skiprows=1,
    header=None,
    usecols=[1, 2, 4, 5, 9, 11],
    names=["Surv", "Classe", "Genre", "Age", "Prix", "Port"],
    dtype={"Surv": object, "Classe": object, "Genre": object, "Port": object},
)
df.head()

In [ ]:
# Définition de variables qualitatives (catégorielles)
df["Surv"] = pd.Categorical(df["Surv"], ordered=False)
df["Classe"] = pd.Categorical(df["Classe"], ordered=False)
df["Genre"] = pd.Categorical(df["Genre"], ordered=False)
df["Port"] = pd.Categorical(df["Port"], ordered=False)
df.dtypes

### 3.1 Échantillonnage simple en lecture

On peut tirer un échantillon aléatoire de lignes au moment de la lecture
d'un gros fichier, en indiquant les lignes à **sauter** (`skiprows`).

In [ ]:
# Exemple d'échantillonnage simple
N = 891   # nombre de lignes du fichier Titanic
n = 200   # taille souhaitée de l'échantillon

lin2skipe = [0]  # ne pas lire la première ligne (en-tête)
lin2skipe.extend(np.random.choice(np.arange(1, N + 1), (N - n), replace=False))

df_small = pd.read_csv(
    path + "titanic-train.csv",
    skiprows=lin2skipe,
    header=None,
    usecols=[1, 2, 4, 5, 9, 11],
    names=["Surv", "Classe", "Genre", "Age", "Prix", "Port"],
)
df_small.head()

## 4. Gérer une table de données

### 4.1 Discrétisation de variables quantitatives (`qcut`, `cut`)

In [ ]:
# Discrétisation en 3 classes d'effectifs proches
df["AgeQ"] = pd.qcut(df.Age, 3, labels=["Ag1", "Ag2", "Ag3"])
df["PrixQ"] = pd.qcut(df.Prix, 3, labels=["Pr1", "Pr2", "Pr3"])
df["PrixQ"].describe()

### 4.2 Recodage / regroupement de modalités

In [ ]:
# Renommage des modalités pour plus de lisibilité
df["Surv"] = df["Surv"].cat.rename_categories(["Vnon", "Voui"])
df["Classe"] = df["Classe"].cat.rename_categories(["Cl1", "Cl2", "Cl3"])
df["Genre"] = df["Genre"].cat.rename_categories(["Gfem", "Gmas"])
df["Port"] = df["Port"].cat.rename_categories(["Pc", "Pq", "Ps"])
df.head()

In [ ]:
# Exemple de recodage par dictionnaire
data = pd.DataFrame(
    {
        "food": [
            "bacon",
            "pulled pork",
            "bacon",
            "Pastrami",
            "corned beef",
            "Bacon",
            "pastrami",
            "honey ham",
            "nova lox",
        ],
        "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6],
    }
)

meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon",
}

data["animal"] = data["food"].map(str.lower).map(meat_to_animal)
data

### 4.3 Variables indicatrices (dummies)


In [ ]:
dfs = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], "data1": range(6)})
pd.get_dummies(dfs["key"])

In [ ]:
dummies = pd.get_dummies(dfs["key"], prefix="key")
df_with_dummy = dfs[["data1"]].join(dummies)
df_with_dummy

### 4.4 Permutation et tirages aléatoires

In [ ]:
dfs = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))
sampler = np.random.permutation(5)
sampler, dfs.take(sampler)

In [ ]:
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)
draws = bag.take(sampler)
draws

### 4.5 Transformations et opérations (`apply`, arithmétique)`

In [ ]:
frame = pd.DataFrame(
    np.random.randn(4, 3),
    columns=list("bde"),
    index=["Utah", "Ohio", "Texas", "Oregon"],
)

f = lambda x: x.max() - x.min()
frame.apply(f, axis=1)

### 4.6 Tri et rangs (`sort_index`, `rank`)

In [ ]:
frame = pd.DataFrame(
    np.arange(8).reshape((2, 4)),
    index=["three", "one"],
    columns=["d", "a", "b", "c"],
)

display(frame.sort_index())
display(frame.sort_index(axis=1))
display(frame.sort_index(axis=1, ascending=False))

In [ ]:
frame = pd.DataFrame(
    {"b": [4.3, 7, -3, 2], "a": [0, 1, 0, 1], "c": [-2, 5, 8, -2.5]}
)

display(frame.rank(axis=1))
display(frame.rank(axis=0))

## 5. Statistiques descriptives élémentaires

On peut utiliser `describe`, `value_counts`, mais aussi tracer des histogrammes,
boxplots, nuages de points, etc.

In [ ]:
import matplotlib.pyplot as plt

df.dtypes

In [ ]:
df.describe(include="all")

In [ ]:
df["Age"].hist()
plt.show()

In [ ]:
df["Prix"].plot(kind="hist")
plt.show()

In [ ]:
df["Surv"].value_counts(), df["Classe"].value_counts(), df["Genre"].value_counts(), df["Port"].value_counts()

### 5.1 Description bivariée

In [ ]:
df.corr(numeric_only=True)

In [ ]:
df.plot(kind="scatter", x="Age", y="Prix")
plt.show()

In [ ]:
df[df["Age"] > 60][["Genre", "Classe", "Age", "Surv"]]

### 5.2 Imputation de données manquantes (approche simple)


In [ ]:
# Remplacement des valeurs manquantes quantitatives par la médiane
df_num = df.select_dtypes(include=["number"])
df[df_num.columns] = df_num.fillna(df_num.median(numeric_only=True))
df.describe(include="all")

## 6. Manipuler des tables de données

### 6.1 Jointures (`merge`)

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], "data1": range(7)})
df2 = pd.DataFrame({"key": ["a", "b", "d"], "data2": range(3)})

display(pd.merge(df1, df2, on="key"))
display(pd.merge(df1, df2, on="key", how="outer"))

### 6.2 Concaténation (`concat`)

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], "var": range(7)})
df2 = pd.DataFrame({"key": ["a", "b", "d"], "var": range(3)})

display(pd.concat([df1, df2], axis=0))
display(pd.concat([df1, df2], axis=1))

## 7. Trafic séquentiel de gros fichiers

Lecture par morceaux (`chunksize`) et stockage au format HDF5 avec
`HDFStore`. **Attention** : la cellule suivante écrit un fichier local `titan.h5`.

In [ ]:
Partition = pd.read_csv(
    path + "titanic-train.csv",
    skiprows=1,
    header=None,
    usecols=[1, 2, 4, 5, 9, 11],
    names=["Surv", "Classe", "Genre", "Age", "Prix", "Port"],
    dtype={"Surv": object, "Classe": object, "Genre": object, "Port": object},
    chunksize=100,
)

store = pd.HDFStore("titan.h5")

for part in Partition:
    # Ici on pourrait insérer du nettoyage / recodage supplémentaire
    store.append("df", part)

store.close()
print("Fichier HDF5 'titan.h5' créé.")

### 7.1 Exemple d'échantillonnage dans un fichier HDF5

On peut ensuite interroger sélectivement la table stockée dans le fichier HDF5.

In [ ]:
archiv = pd.HDFStore("titan.h5")
nrows = archiv.get_storer("df").nrows
r = np.random.randint(0, nrows, size=10)
df_ech = archiv.select("df", where=pd.Index(r))
archiv.close()
df_ech.head()

Ce carnet sert de base pour explorer pandas.